# Exercise 2

**Please Note**: We updated the requirements.txt

Please install the new requirements before editing this exercise.

## Import packages

In [58]:
import os

from vll.utils.download import download_mnist
import numpy as np
import matplotlib.pyplot as plt

import skimage
import skimage.io

import torch
import torch.nn.functional as F
from torchvision import transforms

from models.mnist.simple_cnn import Net

## Task 1
(2 points)

In this task, you will learn some basic tensor operations using the PyTorch library.

Reference for torch: https://pytorch.org/docs/stable/torch.html

In [4]:
# Create a numpy array that looks like this: [0, 1, 2, ..., 19]
arr = np.arange(20)

# Convert the numpy array to a torch tensor
tensor = torch.as_tensor(arr)
print(tensor)

# Create a tensor that contains random numbers.
# It should have the same size like the numpy array.
# Multiply it with the previous tensor.
tensor = tensor * torch.rand(20)
print(tensor)

# Create a tensor that contains only 1s.
# It should have the same size like the numpy array.
# Substract it from the previous tensor.
tensor = tensor - torch.ones(20) 
print(tensor)

# Get the 5th element using a index.
element = torch.select(tensor, -1, 4)
print(element)

# Create a tensor that contains only 0s.
# It should have the same size like the numpy array.
# Multiply it with the previous tensor without any assignment (in place).
print(torch.zeros(20) * tensor)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19])
tensor([ 0.0000,  0.4160,  0.1306,  0.4603,  0.9017,  4.1319,  0.9406,  0.5700,
         4.1559,  3.5557,  6.5802, 10.4088,  6.6508, 11.2416, 10.0554,  7.3762,
         6.9206,  2.7944,  8.2886,  7.1592])
tensor([-1.0000, -0.5840, -0.8694, -0.5397, -0.0983,  3.1319, -0.0594, -0.4300,
         3.1559,  2.5557,  5.5802,  9.4088,  5.6508, 10.2416,  9.0554,  6.3762,
         5.9206,  1.7944,  7.2886,  6.1592])
tensor(-0.0983)
tensor([-0., -0., -0., -0., -0., 0., -0., -0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [5]:
# Load the image from the last exercise as RGB image.
image = np.array(skimage.io.imread('./data/pepo.jpg'))

# Convert the image to a tensor
image = torch.as_tensor(image, dtype=torch.float32)
original_shape = image.shape

# Print its shape
print(image.shape)

# Flatten the image
image = image.view(-1)
print(len(image))

# Add another dimension resulting in a 1x78643 tensor
image = image.unsqueeze(0)
print(image.shape)

# Revert the last action
image = image.squeeze(0)
print(image.shape)

# Reshape the tensor, so that it has the original 2D dimensions
image = image.view(original_shape)
print(image.shape)

# Calculate the sum, mean and max of the tensor
print(torch.sum(image))
print(torch.mean(image))
print(torch.max(image))

torch.Size([512, 512, 3])
786432
torch.Size([1, 786432])
torch.Size([786432])
torch.Size([512, 512, 3])
tensor(91404848.)
tensor(116.2273)
tensor(255.)


## Task 2
(2 points)

Use Autograd to perform operations on a tensor and output then gradients.

In [6]:
# Create a random 2x2 tensor which requires gradients
x = torch.rand(2, requires_grad=True)
print(x)

# Create another tensor by adding 2.0
y = x + 2.0

print(y)

# Create a third tensor z = y^2
z = y ** 2
print(z)

# Compute out as the mean of values in z
out = torch.mean(z)
print(out)

# Perform back propagation on out
out.backward()

# Print the gradients dout/dx
print("Gradients of x:", x.grad)

# Create a copy of y whithout gradients
y2 = y.detach()
print(y2.requires_grad)

# Perform the mean operation on z
# with gradients globally disabled
with torch.no_grad():
    mean_z_no_grad = torch.mean(z)
    print("Mean of z with no gradients:", mean_z_no_grad)

tensor([0.0719, 0.8773], requires_grad=True)
tensor([2.0719, 2.8773], grad_fn=<AddBackward0>)
tensor([4.2929, 8.2791], grad_fn=<PowBackward0>)
tensor(6.2860, grad_fn=<MeanBackward0>)
Gradients of x: tensor([2.0719, 2.8773])
False
Mean of z with no gradients: tensor(6.2860)


## Task 3
(3 points)

Implement a Dataset class for MNIST.

In [7]:
# We first download the MNIST dataset
download_mnist()

MNIST is ready!


In [69]:
class MNIST:
    """
    Dataset class for MNIST
    """

    def __init__(self, root, transform=None):
        """
        root -- path to either "training" or "testing"
        
        transform -- transform (from torchvision.transforms)
                     to be applied to the data
        """
        # save transforms
        self.transform = transform
        
        # create a list of all subdirectories (named like the classes) 
        #       within the dataset root
        subdirs = [os.path.join(root, subdir) for subdir in os.listdir(root) if os.path.isdir(os.path.join(root, subdir))]
        # create a list of paths to all images
        #       with the ground truth label
        self.images = []
        self.labels = []
        for subdir in subdirs:
            images = os.listdir(subdir)
            label = os.path.basename(subdir)
            self.images.extend(list(map(lambda img: os.path.join(subdir, img), images)))
            self.labels.extend([label] * len(images))
        
    def __len__(self):
        """
        Returns the lenght of the dataset (number of images)
        """
        # return the length (number of images) of the dataset
        return len(self.images)
        

    def __getitem__(self, index):
        """
        Loads and returns one image as floating point numpy array
        
        index -- image index in [0, self.__len__() - 1]
        """
        # load the ith image as an numpy array (dtype=float32)
        image_path = self.images[index]
        img = skimage.io.imread(image_path, as_gray=True).astype(np.float32)
        
        # apply transforms to the image (if there are any)
        if self.transform:
            img = self.transform(img)
        
        # return a tuple (transformed image, ground truth)
        return img, self.labels[index]


## Task 4
(3 points)

You can now load a pretrained neural network model we provide.
Your last task is to run the model on the MNIST test dataset, plot some example images with the predicted labels and compute the prediction accuracy.

In [9]:
def validate(model, data_loader):
    # TODO: Create a 10x10 grid of subplots
   
    
    model.eval()
    correct = 0 # count for correct predictions
    
    with torch.no_grad():
        for i, item in enumerate(data_loader):
            # TODO: unpack item into image and ground truth
            #       and run network on them
            
            
            # TODO: get class with highest probability
            
            
            # TODO: check if prediction is correct
            #       and add it to correct count
            
            
            # plot the first 100 images
            if i < 100:
                # TODO: compute position of ith image in the grid
                
                
                # TODO: convert image tensor to numpy array
                #       and normalize to [0, 1]
                
                
                # TODO: make wrongly predicted images red
                
                
                # TODO: disable axis and show image
                
                
                # TODO: show the predicted class next to each image
                
            
            elif i == 100:
                plt.show()
    
    # TODO: compute and print the prediction accuracy in percent
    

# create a DataLoader using the implemented MNIST dataset class
data_loader = torch.utils.data.DataLoader(
    MNIST('data/mnist/testing',
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=1, shuffle=True)

# create the neural network
model = Net()

# load the statedict from 'models/mnist/simple_cnn.pt'
model.load_state_dict(torch.load('models/mnist/simple_cnn.pt'))

# validate the model
validate(model, data_loader)

IndentationError: expected an indented block after 'if' statement on line 22 (2882649406.py, line 39)